# Group 12 Coursework 

In [1]:
import os
import datetime
import time
import json
import http.client
import requests
from html.parser import HTMLParser
from html.entities import name2codepoint


class MyHTMLParser(HTMLParser):
# This is a class for parsing the html returned from the web service endpoint

    def __init__(self):
        super(MyHTMLParser, self).__init__()
        self.emotion_type = None
        self.emotion_return = False
        
    def handle_starttag(self, tag, attrs):
        if tag=='h3':
            self.emotion_return = True
    
    def handle_data(self, data):
        if self.emotion_return:
            self.emotion_type = data.strip()
        
    def handle_endtag(self, tag):
        if tag=='h3':
            self.emotion_return = False

def deploy_changes(commitmsg):
    '''Run git commands to deploy current changes in project folder to Github and deploy to web server'''
    
    !git add .
    !git commit -m commitmsg
    !git push origin master
    print('Changes were deployed to Github. Please action the CI/CD pipeline running status.')
    
    

### 1. Submit user input for classifying emotion
Below cell will read user input text, and then make a HTTP request using POST method to the webservice endpoint deployed locally (http://127.0.0.1:5000) to classify the emotion of the user input text.

In [2]:
usertext = input("Please input your text:  ")

Please input your text:  sorry guys


In [3]:
data = {'input_text': usertext}
dbipaddr = '127.0.0.1:5000'

response = requests.post('http://' + dbipaddr, data=data)

parser = MyHTMLParser()

if response.status_code==200:       
    parser.feed(response.text)
    if parser.emotion_type is not None:
        print(f'The classified emotion type is: {parser.emotion_type}')
    else:
        print(f'No emotion can be classified!')

The classified emotion type is: disappointment_embarrassment_grief_remorse_sadness


### 2. Apply code changes to CI/CD Pipeline in Github

The CI/CD pipeline is setup in Github. The webservice endpoint, which is deployed in a local Ubuntu VM, is self-host runner. A CI/CD listener is running continuously in the VM to listen to code changes in the Github repository. Any new code changes or model file changes pushed to the repository will be downloaded and applied to the webservice endpoint.

In [4]:
deploy_changes("updated run_app.ipynb to include the performance test.")

On branch master
nothing to commit, working tree clean
Changes were deployed to Github. Please action the CI/CD pipeline running status.


Everything up-to-date


### 3. Get log entry from web server for monitoring

The webservice endpoint will record the user submitted text, the classification result as well as the time taken to run the classification task in the web server. Below cell requests the web server to retrieve the last 7 days logs and parsed it as simple tabular data output.

In [5]:
response = requests.get('http://' + dbipaddr + '/getlog')

# parser = MyHTMLParser()

if response.status_code==200:
    lines = response.text.split('</p>')
    for l in lines:    
        print(l[3:])
 

input_run_time,input_text,end_run_time,result
'2023-05-21 20:48:25.736870','sorry guys','2023-05-21 20:48:25.944417','disappointment_embarrassment_grief_remorse_sadness'
'2023-05-21 20:48:44.845683','sorry guys','2023-05-21 20:48:44.993890','disappointment_embarrassment_grief_remorse_sadness'



### 4. Performance test

In [6]:
#performance test script
import requests
from concurrent.futures import ThreadPoolExecutor

In [7]:
# send the single request
url = 'http://localhost:5000'
res = requests.get(url)

data = "input_text=The king is good"
headers = {'Content-Type': 'application/x-www-form-urlencoded'}

res = requests.post(url=url, data=data, headers=headers)
time_elapsed = res.elapsed.total_seconds()

print(res)
print(f"{res.text}")
print('time_elapsed:', time_elapsed)

<Response [200]>
<html>
    <body>
        
          <p> 
            <h1 style="font-size:1.2em">
               Input text for classification is below:
            </h1>
         </p>
          <p style="color:blue">The king is good </p>
          <p> 
            <h2 style="font-size:1.2em">
               The predicted emotion is below.
            </h2>
         </p>
          <p>
            <h3 style="color:red">
               admiration 
            </h3>
         </p>
        
        input the text
        <form action = "./" method = "POST" enctype = "multipart/form-data">
           <input type="text" name="input_text" size="200" value="input the text"></p>
           <input type = "submit"/>
        </form>
     </body>
</html>
time_elapsed: 2.144029


In [8]:
# make the text with arbitrary length
def makeText(n):
    message="The king is good. "   
    text = [message[i % len(message)] for i in range(n)]
    return "input_text="+''.join(text)

In [9]:
print(makeText(30))
print(makeText(60))

input_text=The king is good. The king is 
input_text=The king is good. The king is good. The king is good. The ki


In [12]:
# Define the function for POST message
def getEmotion(id,message):
    #print("id:",str(id))
    try:
        url = 'http://localhost:5000'
        res = requests.get(url)
        headers = {'Content-Type': 'application/x-www-form-urlencoded'}
        res = requests.post(url=url, data=message, headers=headers,timeout=300)
    except IOError:
        print("Failed to open",host)
    return res.elapsed.total_seconds()

In [ ]:
concurrent_num = [1,10,50,100] #
text_length = [10,100,300,500,1000]
elapsed_times=[[0]*len(text_length) for i in range(len(concurrent_num))]

for i in concurrent_num:
    for j in text_length:
        runs=[value for value in range(i)]
        message = makeText(j)
        message = [message]*i
        with ThreadPoolExecutor(max_workers=i) as executor:
            results = executor.map(getEmotion,runs,message)
        elapsed_time = []
        for result in results:
            elapsed_time.append(result)
        #print("concurrent_num: "+str(i)+", text_length: "+str(j))
        #print("concurrent_num: "+str(concurrent_num.index(i))+", text_length: "+str(text_length.index(j)))
        #print(elapsed_time)
        elapsed_times[concurrent_num.index(i)][text_length.index(j)]=sum(elapsed_time)/len(elapsed_time)

In [ ]:
import matplotlib.pyplot as plt

plt.xlabel("concurrent_num", fontsize=16)
plt.ylabel("elapsed_time", fontsize=16)
plt.grid(True)
plt.tick_params(labelsize = 12) 
plt.plot(concurrent_num, [row[0] for row in elapsed_times], label="text_length=10")
plt.plot(concurrent_num, [row[1] for row in elapsed_times], label="text_length=100")
plt.plot(concurrent_num, [row[2] for row in elapsed_times], label="text_length=300")
plt.plot(concurrent_num, [row[3] for row in elapsed_times], label="text_length=500")
plt.plot(concurrent_num, [row[4] for row in elapsed_times], label="text_length=1000")
plt.legend(fontsize=12)
plt.show()

In [ ]:
# Many concurrent number causes error
# This is because the server has received so many requests that it cannot process them all, 
# Therefore, requests won't receive a response.
concurrent_num = 500
text_length = 100

runs=[value for value in range(concurrent_num)]
message = makeText(text_length)
message = [message]*concurrent_num

with ThreadPoolExecutor(max_workers=concurrent_num) as executor:
    results = executor.map(getEmotion,runs,message)

In [ ]:
elapsed_times = []
for result in results:
    elapsed_times.append(result)
    #print(result.elapsed.total_seconds())
print(len(elapsed_times))
print(sum(elapsed_times)/len(elapsed_times))